In [55]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd

In [56]:
#========
# Setup
#========
ts = pd.DataFrame({
    "date": pd.date_range("2025-11-01" , periods = 14 , freq = "D").tolist() * 2 ,
    "region": ["East"] * 14 + ["West"] * 14 ,
    "sales": [110 , 108 , 115 , 120 , 118 , 125 , 130 , 128 , 132 , 135 , 133 , 138 , 140 , 142] + [95 ,  98 , 100 , 102 ,  99 , 105 , 108 , 107 , 109 , 111 , 110 , 112 , 115 , 117] ,
    "returns": [4 , 5 , 4 , 6 , 5 , 6 , 7 , 6 , 5 , 7 , 6 , 8 , 7 , 6] + [3 , 2 , 3 , 4 , 3 , 4 , 5 , 4 , 4 , 5 , 4 , 5 , 6 , 5] ,
    "margin": [0.18 , 0.17 , 0.19 , 0.20 , 0.19 , 0.21 , 0.22 , 0.21 , 0.23 , 0.24 , 0.23 , 0.25 , 0.26 , 0.26] + [0.12 , 0.13 , 0.12 , 0.14 , 0.13 , 0.15 , 0.16 , 0.15 , 0.16 , 0.17 , 0.16 , 0.17 , 0.18 , 0.18] ,
})
ts = ts.sort_values(["region" , "date"]).reset_index(drop = True)
ts.head(10)

,date,region,sales,returns,margin
0,2025-11-01,East,110,4,0.18
1,2025-11-02,East,108,5,0.17
2,2025-11-03,East,115,4,0.19
3,2025-11-04,East,120,6,0.20
4,2025-11-05,East,118,5,0.19
5,2025-11-06,East,125,6,0.21
6,2025-11-07,East,130,7,0.22
7,2025-11-08,East,128,6,0.21
8,2025-11-09,East,132,5,0.23
9,2025-11-10,East,135,7,0.24


In [57]:
#===============================
# Case 1) Rolling correlation
#===============================
ts["roll_corr_sales_returns"] = (
    ts.groupby("region" , group_keys = False)[["sales" , "returns"]]
        .apply(lambda g: g["sales"].rolling(window = 5 , min_periods = 4).corr(g["returns"]))
)
ts.loc[: , ["region" , "date" , "sales" , "returns" , "roll_corr_sales_returns"]].head(10).round(3)

,region,date,sales,returns,roll_corr_sales_returns
0,East,2025-11-01,110,4,NaN
1,East,2025-11-02,108,5,NaN
2,East,2025-11-03,115,4,NaN
3,East,2025-11-04,120,6,0.534
4,East,2025-11-05,118,5,0.537
5,East,2025-11-06,125,6,0.607
6,East,2025-11-07,130,7,0.930
7,East,2025-11-08,128,6,0.829
8,East,2025-11-09,132,5,0.361
9,East,2025-11-10,135,7,0.235


In [58]:
#==============================
# Case 2) Rolling covariance
#==============================
ts["roll_cov_sales_returns"] = (
    ts.groupby("region" , group_keys = False)[["sales" , "returns"]]
        .apply(lambda g: g["sales"].rolling(window = 5 , min_periods = 4).cov(g["returns"]))
)
ts.loc[: , ["region" , "date" , "roll_cov_sales_returns"]].head(10).round(3)

,region,date,roll_cov_sales_returns
0,East,2025-11-01,NaN
1,East,2025-11-02,NaN
2,East,2025-11-03,NaN
3,East,2025-11-04,2.75
4,East,2025-11-05,2.30
5,East,2025-11-06,3.20
6,East,2025-11-07,6.30
7,East,2025-11-08,3.00
8,East,2025-11-09,1.65
9,East,2025-11-10,0.75


In [59]:
#======================================================
# Case 3) Rolling correlation matrix (multiple KPIs)
#======================================================
east = ts.loc[ts["region"].eq("East")].set_index("date")[["sales" , "returns" , "margin"]]

roll_corr_matrix = east.rolling(window = 5 , min_periods = 4).corr()
roll_corr_sales_margin = roll_corr_matrix.xs("sales" , level = 1)["margin"]
roll_corr_sales_margin.dropna().tail(6).round(3)

date
2025-11-09    0.970
2025-11-10    0.957
2025-11-11    0.990
2025-11-12    0.993
2025-11-13    0.992
2025-11-14    0.978
Name: margin, dtype: float64

In [60]:
#==========================
# Case 4) EWM correlation
#==========================
ts["ewm_corr_sales_returns"] = (
    ts.groupby("region" , group_keys = False)[["sales" , "returns"]]
        .apply(lambda g: g["sales"].ewm(span = 8 , adjust = False).corr(g["returns"]))
)
ts.loc[: , ["region" , "date" , "ewm_corr_sales_returns"]].head(10).round(3)

,region,date,ewm_corr_sales_returns
0,East,2025-11-01,NaN
1,East,2025-11-02,-1.000
2,East,2025-11-03,-0.531
3,East,2025-11-04,0.701
4,East,2025-11-05,0.709
5,East,2025-11-06,0.840
6,East,2025-11-07,0.926
7,East,2025-11-08,0.907
8,East,2025-11-09,0.604
9,East,2025-11-10,0.737


In [61]:
#==========================
# Case 5) EWM covariance
#==========================
ts["ewm_cov_sales_returns"] = (
    ts.groupby("region" , group_keys = False)[["sales" , "returns"]]
        .apply(lambda g: g["sales"].ewm(span = 8 , adjust = False).cov(g["returns"]))
)
ts.loc[: , ["region" , "date" , "ewm_cov_sales_returns"]].head(10).round(3)

,region,date,ewm_cov_sales_returns
0,East,2025-11-01,NaN
1,East,2025-11-02,-1.000
2,East,2025-11-03,-0.862
3,East,2025-11-04,3.735
4,East,2025-11-05,3.109
5,East,2025-11-06,5.427
6,East,2025-11-07,9.812
7,East,2025-11-08,8.491
8,East,2025-11-09,5.350
9,East,2025-11-10,7.680


In [62]:
#=============================================================
# Case 6) Performance + method="table" (multi-column window)
#=============================================================
def stability_score(arr: np.ndarray) -> float:
    sales = arr[: , 0]
    rets = arr[: , 1]
    return float(np.std(sales) + 10 * np.std(rets))

tmp = ts.loc[ts["region"].eq("East") , ["sales" , "returns"]]
score = (
    tmp.rolling(window = 5 , min_periods = 4 , method = "table")
        .apply(stability_score , raw = True , engine = "numba")
)
score.tail(6)

,sales,returns
8,12.365937,12.365937
9,10.889192,10.889192
10,9.899924,9.899924
11,13.508628,13.508628
12,13.204698,13.204698
13,10.745216,10.745216
